In [73]:
!python -m pip install --upgrade pip
!pip install tensorflow
!pip install pandas

     -------------------------------------- 444.1/444.1 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 14.2/14.2 MB 9.8 MB/s eta 0:00:00
     -------------------------------------- 895.5/895.5 kB 9.4 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     -------------------------------------- 438.7/438.7 kB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 10.3 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 10.4 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 10.8 MB/s eta 0:00:00
     ------------------

In [69]:
import pandas as pd

df = pd.read_csv("Bitcoin_Trainingdata.csv")

#get percentage change in Open values
df["Open_previous"] = df["Open"].shift(1)
df["Change_in_Open"] = (df["Open"] - df["Open_previous"]) / df["Open_previous"]
df = df.dropna()

df.tail()

,Date,Open,High,Low,Close,Volume,Currency,Open_previous,Change_in_Open
2787,2022-05-05,39695.746094,39789.281250,35856.515625,36575.140625,43106256317,USD,37748.011719,0.051598
2788,2022-05-06,36573.183594,36624.359375,35482.132812,36040.921875,37795577489,USD,39695.746094,-0.078662
2789,2022-05-07,36042.503906,36129.925781,34940.824219,35501.953125,24375896406,USD,36573.183594,-0.014510
2790,2022-05-08,35502.941406,35502.941406,33878.964844,34059.265625,36763041910,USD,36042.503906,-0.014970
2791,2022-05-09,34039.796875,34210.621094,31337.292969,31425.814453,49328988160,USD,35502.941406,-0.041212


In [77]:
import numpy as np

X = []
Y = []

changes = df["Change_in_Open"]

for i in range(len(changes), 20, -1):
    Y.append(changes[i])
    X.append(np.array(changes[i-21:i-1]))
    
X = np.array(X).reshape(-1, 20, 1)
Y = np.array(Y)

In [78]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM

model = Sequential()
model.add(LSTM(1, input_shape=(20, 1)))

model.compile(optimizer="rmsprop", loss="mse")
model.fit(X, Y, batch_size=32, epochs=10)

Epoch 1/10
87/87 [==============================] - 4s 16ms/step - loss: 0.0015
Epoch 2/10
87/87 [==============================] - 1s 16ms/step - loss: 0.0015
Epoch 3/10
87/87 [==============================] - 1s 15ms/step - loss: 0.0015
Epoch 4/10
87/87 [==============================] - 1s 14ms/step - loss: 0.0015
Epoch 5/10
87/87 [==============================] - 1s 15ms/step - loss: 0.0015
Epoch 6/10
87/87 [==============================] - 1s 15ms/step - loss: 0.0015
Epoch 7/10
87/87 [==============================] - 1s 14ms/step - loss: 0.0015
Epoch 8/10
87/87 [==============================] - 1s 16ms/step - loss: 0.0015
Epoch 9/10
87/87 [==============================] - 1s 15ms/step - loss: 0.0015
Epoch 10/10
87/87 [==============================] - 2s 21ms/step - loss: 0.0015


In [79]:
model.predict(X)

array([[ 0.00393416],
       [ 0.004441  ],
       [ 0.0056711 ],
       ...,
       [-0.00194454],
       [ 0.00072051],
       [ 0.00191879]], dtype=float32)